In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import ExtraTreesClassifier,GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from nltk.stem import SnowballStemmer,PorterStemmer
from stemming import lovins,porter2,paicehusk
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import stop_words
import pprint
import re
import time
import spacy
import datetime
import requests
import BeautifulSoup
%matplotlib inline

In [2]:
### THIS CODE LOADS THE JSONS OF THE VECTORIZED REDDIT COMMENTS, AND GETS IT TO ITS PRIOR USABLE STATE
total_time=time.time()
print "Starting Reddit DF Loading"
start_time=time.time()
news=pd.read_json('../r_news.json')
worldnews=pd.read_json('../r_worldnews.json')
tech=pd.read_json('../r_technology.json')
stock=pd.read_json('../r_StockMarket.json')
politics=pd.read_json('../r_politics.json')
inthenews=pd.read_json('../r_inthenews.json')
futurology=pd.read_json('../r_Futurology.json')
dfs={'news':news,'worldnews':worldnews,'tech':tech,'stock':stock,'politics':politics,\
     'inthenews':inthenews,'futurology':futurology}
for name in dfs.keys():
    df=dfs[name]
    for i in range(len(df.columns)):
        all_words=pprint.pformat(df.iloc[4,i])
        all_words=all_words.replace("\n"," ").replace("\\n"," ").replace("{u'comment': ","")\
                .replace(",","").replace("ucomment: ","")
        all_words=re.sub(r"[\[\]]+"," ",all_words)
        all_words=re.sub(r"\Wu\'id\': \w\'\w{7}\'","",all_words).replace("{","").replace("}","").replace("      "," ")\
                    .replace("\\\'","\'").replace("' u\'"," ").replace(" u\'","")
        all_words=re.sub(r"\\u20..","",all_words).replace("\'","").replace(' u"',"")
        all_words=re.sub(r"[A-Za-z]*[0-9]+[A-Za-z]*","",all_words).replace("*","")
        df.iloc[4,i]=all_words
    dfs[name]=df
print "Finished Reddit DF Loading"
print "Took %s Minutes"%((time.time()-start_time)/60.)
print 

jsons=[]
json_dfs=[]
print "Starting Vectorized Words Loading"
start_time=time.time()
for link in ['../vectorized_r_news.json','../vectorized_r_worldnews.json','../vectorized_r_technology.json',\
            '../vectorized_r_StockMarket.json','../vectorized_r_politics.json','../vectorized_r_inthenews.json',\
            '../vectorized_r_Futurology.json']:
    jsons.append(pd.read_json(link))

start_time=time.time()
for i in range(len(jsons)):
    temp_df=jsons[i]
    temp_df=temp_df.T.sort_values("Mean",ascending=False)
    del temp_df["Mean"]
    new_cols=[]
    for col in temp_df.columns:
        new_cols.append(int(col))
    temp_df.columns=new_cols
    temp_df["Mean"]=temp_df.T.mean()
    temp_df=temp_df.sort_values("Mean",ascending=False).T.sort_index()
    json_dfs.append(temp_df)
print "Finished Vectorized Words Loading"
print "Took: %s Minutes" %((time.time()-start_time)/60.)
print

###---

start_time=time.time()
print "Starting Fed, Keywords, and Pos/Neg"
fed=pd.read_json('../fed_info.json')
del fed['f2015043']
del fed['f2015042']
cols=[]
for col in fed.columns:
    if col[0]!='f':
        name=col[:4]+'-'+col[4:6]+'-'+col[6:]
        cols.append(name)
    else: print col
cols=pd.to_datetime(cols)
fed.columns=cols
# We eliminated columns before the 119th because we want to focus in on a 1 year period. We state the end of the 
# period is the last date in our data set
fed=fed.iloc[:,119:]
keywords=pd.read_json('../key_terms.json').reset_index()
keywords['index']=keywords['index'].astype(int)
keywords=keywords.sort_values('index')
keywords.index=keywords['index']
del keywords['index']
keywords.columns=['Type','Value']
keywords["Value"]=keywords["Value"].apply(lambda x:x.lower())

posneg=pd.read_csv('../subjectivity of words.txt')
word_list=['type=weaksubj len=1 word1=abandoned pos1=adj stemmed1=n priorpolarity=negative'.split(" ")]
posneg.columns=['col']
for row in posneg['col']:
    word_list.append(row.split(" "))
posneg=[]
for x in word_list:
    posneg.append([])
    for y in x:
        posneg[-1].append(y.split("=")[-1])
posneg=pd.DataFrame(posneg)
posneg.columns=['type','len','word','pos','stemmed','priorpolarity','N/A']
print "Finished Fed, Keywords, Pos/Neg"
print "Took: %s Minutes" %((time.time()-start_time())/60.)
print "\n \n"
print "Total time: %s Minutes"%((time.time()-total_time)/60.)

Starting JSON Loading
Loading JSON at Time: 1.58150990804e-06 Minutes
Loading JSON at Time: 0.955058598518 Minutes
Loading JSON at Time: 2.19378606478 Minutes
Loading JSON at Time: 3.52724283139 Minutes
Loading JSON at Time: 3.82851681709 Minutes
Loading JSON at Time: 4.97993968328 Minutes
Loading JSON at Time: 6.52209508419 Minutes
Finished JSON Loading
Took: 7.38683884939 Minutes

Starting DF reformatting
Finished DF reformatting
Took: 0.862464189529 Seconds


In [179]:
print "Starting Reddit DF Loading"
start_time=time.time()
news=pd.read_json('../r_news.json')
worldnews=pd.read_json('../r_worldnews.json')
tech=pd.read_json('../r_technology.json')
stock=pd.read_json('../r_StockMarket.json')
politics=pd.read_json('../r_politics.json')
inthenews=pd.read_json('../r_inthenews.json')
futurology=pd.read_json('../r_Futurology.json')
dfs={'news':news,'worldnews':worldnews,'tech':tech,'stock':stock,'politics':politics,\
     'inthenews':inthenews,'futurology':futurology}
for name in dfs.keys():
    df=dfs[name]
    for i in range(len(df.columns)):
        all_words=pprint.pformat(df.iloc[4,i])
        all_words=all_words.replace("\n"," ").replace("\\n"," ").replace("{u'comment': ","")\
                .replace(",","").replace("ucomment: ","")
        all_words=re.sub(r"[\[\]]+"," ",all_words)
        all_words=re.sub(r"\Wu\'id\': \w\'\w{7}\'","",all_words).replace("{","").replace("}","").replace("      "," ")\
                    .replace("\\\'","\'").replace("' u\'"," ").replace(" u\'","")
        all_words=re.sub(r"\\u20..","",all_words).replace("\'","").replace(' u"',"")
        all_words=re.sub(r"[A-Za-z]*[0-9]+[A-Za-z]*","",all_words).replace("*","")
        df.iloc[4,i]=all_words
    dfs[name]=df
print "Finished Reddit DF Loading"
print "Took:",time.time()-start_time

Starting Reddit DF Loading
Finished Reddit DF Loading
Took: 86.6783220768


Create a dictionary of all the words, and link the word to a keyword identity and strength. 
It might be smart to take a series of documents and find out how often each of the keysords appears. From this, one could deduce an implied strength by their relevance in the articles. Financial articles would be the best choice for this. (Maybe scrape Yahoo finance?)

Categorize the post to the identities and give them a strength.
    
* There would be a column for each type of keyword. The value of the column would be the sum of the keywords with that category. The algorithms would weight each of those values linearly, so scale them appropriately before. This would be done just after the vectorization process.
    
The next step is to introduce the sentiment implied by the comments and the direction of this sentiment. It might be smart to use only the top posts, and none of their child comments (they can get pretty stupid.) However, there are some subreddits with sane commentors; we wouldn't necessarily need to filter these (especially as they will probably have fewer comments).

After the vectorized sentiment has been calculated, we will have keywords to identify them with. If the keywords in the comments match what is in the title, then it is safe to assume an implied investor sentiment; this will be a column. (Here, it might be smart to create derivative columns consisting of the implied sentiment column and the identity columns).

From here, we can begin to look at classifications, regressions, and clustering.

We want to predict $\Delta$Price, and reset the price on the day after we think it is going to be realized.

Go through all of the fed articles and see:

    1) How many times each type of keyword occurs 
    2) How many times each keyword appears

In [175]:
type_dict_pos={}
for word in keywords['Type']:
    type_dict_pos[word]=0
type_dict_neg=type_dict_pos.copy()
type_dict_gen=type_dict_pos.copy()
word_dict_pos={}
for word in keywords['Value']:
    word_dict_pos[word]=0
word_dict_neg=word_dict_pos.copy()
word_dict_gen=word_dict_pos.copy()



for col in fed.columns:
    title=fed.loc[0,col]
    pos_count=0
    neg_count=0
    for word in posneg['word']:
        if word in title:
            sentiment=posneg[posneg['word']==word]['priorpolarity'].values[0]
            if sentiment=='negative':
                neg_count+=1
            elif sentiment=='positive':
                pos_count+=1
    article=""
    for x in fed.loc[2,col]:
        article+=x.lower()+' '
    if pos_count>neg_count:
        for word in keywords['Value']:
            if word in article:
                word_dict_pos[word]+=1
                type_dict_pos[keywords[keywords['Value']==word]['Type'].values[0]]+=1
    elif pos_count<neg_count:
        for word in keywords['Value']:
            if word in article:
                word_dict_neg[word]+=1
                type_dict_neg[keywords[keywords['Value']==word]['Type'].values[0]]+=1
    else:
        for word in keywords['Value']:
            if word in article:
                word_dict_gen[word]+=1
                type_dict_gen[keywords[keywords['Value']==word]['Type'].values[0]]+=1

In [165]:
word_results=pd.DataFrame([word_dict_neg,word_dict_pos,word_dict_gen]).T
word_results.columns=['Negative','Positive',"General"]
word_results["Occurences"]=word_results["Positive"]+word_results["Negative"]+word_results["General"]
word_results["Rating"]=(word_results["Positive"]-word_results['Negative'])/word_results["Occurences"]
word_results[word_results['Occurences']>3].sort_values("Rating")[["Occurences","Rating"]]

In [174]:
word_results[word_results['Occurences']>3].sort_values("Rating")[["Occurences","Rating"]]

Occurences    Rating
iowa                             8 -1.000000
georgia                         40 -1.000000
indiana                         10 -1.000000
maryland                         4 -1.000000
massachusetts                   12 -1.000000
minnesota                       16 -1.000000
columbus                         4 -1.000000
florida                         69 -1.000000
california                     518 -1.000000
ohio                            14 -1.000000
missouri                        24 -1.000000
arizona                         11 -1.000000
alabama                         10 -1.000000
stock                            4 -1.000000
oklahoma                        10 -1.000000
kansas                          35 -0.800000
virginia                        40 -0.800000
san francisco                    4 -0.750000
trading                          8 -0.750000
city                             5 -0.600000
exchange                         7 -0.571429
kansas city                      4 -0.500000
finance                          6 -0.500000
government                      14 -0.428571
chicago                          5 -0.400000
accepted                         5 -0.400000
department                       5 -0.400000
million                          8 -0.375000
nation                         120 -0.366667
media                          118 -0.347458
...                            ...       ...
bankrupt                         4  0.000000
bankruptcy                       4  0.000000
equity                           8  0.000000
rice                            11  0.000000
rates                            5  0.000000
implementation                  13  0.000000
pennsylvania                    16  0.000000
price                           11  0.000000
paid                             6  0.000000
owed                             7  0.000000
inflation                       14  0.000000
colorado                        26  0.000000
debt                            12  0.000000
columbia                         6  0.000000
federal register                48  0.083333
unity                           19  0.105263
business                        18  0.111111
liquidity                        6  0.166667
texas                          114  0.333333
economy                          5  0.400000
authority                        6  0.500000
loan                             8  0.500000
gold                             4  0.500000
goldman sachs                    4  0.500000
federal reserve system           6  0.500000
north carolina                  40  0.500000
loans                            5  0.600000
connecticut                      6  1.000000
utah                             5  1.000000
louisiana                        5  1.000000

[98 rows x 2 columns]

In [200]:
type_dict_pos={}
for word in keywords['Type']:
    type_dict_pos[word]=0
type_dict_neg=type_dict_pos.copy()
type_dict_gen=type_dict_pos.copy()
word_dict_pos={}
for word in keywords['Value']:
    word_dict_pos[word]=0
word_dict_neg=word_dict_pos.copy()
word_dict_gen=word_dict_pos.copy()


article_row=4
for name in dfs.keys():
    df=dfs[name]
    for col in df.columns:
        title=df.loc[0,col]
        pos_count=0
        neg_count=0
        for word in posneg['word']:
            if word in title:
                sentiment=posneg[posneg['word']==word]['priorpolarity'].values[0]
                if sentiment=='negative':
                    neg_count+=1
                elif sentiment=='positive':
                    pos_count+=1
    # Use for fed
    #     article=""
    #     for x in df.loc[article_row,col]:
    #         article+=x.lower()+' '
        article=df.loc[article_row,col].lower()
        if pos_count>neg_count:
            for word in keywords['Value']:
                if word in article:
                    word_dict_pos[word]+=1
                    type_dict_pos[keywords[keywords['Value']==word]['Type'].values[0]]+=1
        elif pos_count<neg_count:
            for word in keywords['Value']:
                if word in article:
                    word_dict_neg[word]+=1
                    type_dict_neg[keywords[keywords['Value']==word]['Type'].values[0]]+=1
        else:
            for word in keywords['Value']:
                if word in article:
                    word_dict_gen[word]+=1
                    type_dict_gen[keywords[keywords['Value']==word]['Type'].values[0]]+=1

In [201]:
word_results=pd.DataFrame([word_dict_neg,word_dict_pos,word_dict_gen]).T
word_results.columns=['Negative','Positive',"General"]
word_results["Occurences"]=word_results["Positive"]+word_results["Negative"]+word_results["General"]
word_results["Rating"]=(word_results["Positive"]-word_results['Negative'])/word_results["Occurences"]
word_results[word_results['Occurences']>15].sort_values("Rating")[["Occurences","Rating"]]

Occurences    Rating
federal election commission          18 -0.888889
halliburton                          31 -0.806452
tucson                               21 -0.761905
limited government                   20 -0.750000
environmentalism                     24 -0.708333
kroger                               25 -0.680000
open primary                         24 -0.666667
closed primary                       26 -0.653846
deceived                             25 -0.640000
berkeley                             30 -0.633333
louisiana                           510 -0.627451
san bernardino                       58 -0.620690
norman                               18 -0.611111
tyler                                18 -0.611111
individualism                        18 -0.611111
biomet                               23 -0.608696
national convention                  35 -0.600000
fiscal year                          42 -0.595238
civil service                        32 -0.593750
yielded                              22 -0.590909
raleigh                              17 -0.588235
roe v. wade                          17 -0.588235
craigslist                           46 -0.586957
kbr                                  19 -0.578947
public policy                        54 -0.574074
federalism                           21 -0.571429
gag order                            42 -0.571429
charlotte                            37 -0.567568
delaware                             67 -0.567164
burger king                          25 -0.560000
...                                 ...       ...
eureka                               25 -0.200000
waco                                 20 -0.200000
salt lake city                       21 -0.190476
businessmen                          63 -0.190476
speaker of the house                 21 -0.190476
xerox                                16 -0.187500
san jose                             22 -0.181818
bilateral                            17 -0.176471
anarchism                            17 -0.176471
hot topic                            29 -0.172414
bundling                             35 -0.171429
tulsa                                18 -0.166667
richmond                             36 -0.166667
antec                                31 -0.161290
plurality                            45 -0.155556
minneapolis                          46 -0.152174
converse                             93 -0.139785
pollster                             36 -0.138889
cambridge                            37 -0.135135
mesa                                 38 -0.131579
nation-state                         16 -0.125000
orlando                              74 -0.121622
lockheed martin                      27 -0.111111
guam                                 18 -0.111111
edison                               42 -0.071429
fiscal policy                        18 -0.055556
welfare state                        48 -0.041667
fairness doctrine                    20  0.000000
manchester                           21  0.047619
boulder                              29  0.172414

[533 rows x 2 columns]